In [53]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import  pyspark.sql.functions as fun 
import numpy as np
from pyspark.sql.types import *

In [54]:
spark = SparkSession.builder.getOrCreate()

In [55]:
sc = spark.sparkContext

Create an RDD from a list of numbers (1,50) using numpy methods

In [56]:
numbers_rdd = sc.parallelize(np.arange(1,50))
print(numbers_rdd.collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


Find the sum, average, maximum, minimum, and count

In [57]:
#Sum
numbers_rdd.sum()

1225

In [58]:
#avg
numbers_rdd.mean()

25.0

In [59]:
#Count
numbers_rdd.count()

49

In [60]:
#Min
numbers_rdd.min()

1

In [61]:
#Max
numbers_rdd.max()

49

Count how many numbers are even vs. odd.

In [62]:
even_count = numbers_rdd.filter(lambda x: x % 2 == 0).count()
odd_count = numbers_rdd.filter(lambda x: x % 2 != 0).count()
print(f"ODD Numbers: {odd_count} EVEN Numbers: {even_count}")

ODD Numbers: 25 EVEN Numbers: 24


You have the following data of people info ('Name', 'Age'), answer the following questions

In [63]:
people_data = [("Nada ", 25), ("Mona", 30), ("Ahmed", 35), ("Khaled", 40),("Ahmed", 35), ('Nada ', 25)]
rdd_people = sc.parallelize(people_data)
rdd_people.collect()

[('Nada ', 25),
 ('Mona', 30),
 ('Ahmed', 35),
 ('Khaled', 40),
 ('Ahmed', 35),
 ('Nada ', 25)]

Find the oldest person

In [64]:
rdd_people.max(key=lambda x: x[1])

('Khaled', 40)

Compute the average age

In [65]:
rdd_people.map(lambda x: x[1]).mean()

31.666666666666668

Group all the names by their age

In [66]:
rdd_people.groupBy(lambda x: x[1]).mapValues(list).collect()

[(25, [('Nada ', 25), ('Nada ', 25)]),
 (40, [('Khaled', 40)]),
 (30, [('Mona', 30)]),
 (35, [('Ahmed', 35), ('Ahmed', 35)])]

Take the following text and put it in a text file named russia.txt and load it into rdd

"Russia is the largest country in the world by land area
Moscow is the capital city of Russia
The Russian language is one of the most widely spoken languages in the world
Russia is known for its rich history and culture
The Trans-Siberian Railway is the longest railway line in the world
Russia has a strong tradition in literature, music and ballet
The country is famous for its cold winters and vast landscapes
Russia is a major player in global energy production
"


In [67]:
russia_text =[
"Russia is the largest country in the world by land area",
"Moscow is the capital city of Russia",
"The Russian language is one of the most widely spoken languages in the world",
"Russia is known for its rich history and culture",
"The Trans-Siberian Railway is the longest railway line in the world",
"Russia has a strong tradition in literature, music and ballet",
"The country is famous for its cold winters and vast landscapes",
"Russia is a major player in global energy production",

]



In [68]:
rdd_russia = sc.parallelize(russia_text)
rdd_russia.collect()

['Russia is the largest country in the world by land area',
 'Moscow is the capital city of Russia',
 'The Russian language is one of the most widely spoken languages in the world',
 'Russia is known for its rich history and culture',
 'The Trans-Siberian Railway is the longest railway line in the world',
 'Russia has a strong tradition in literature, music and ballet',
 'The country is famous for its cold winters and vast landscapes',
 'Russia is a major player in global energy production']

In [69]:
# rdd_russia = sc.textFile("/data/russia.txt")
# rdd_russia.collect()

Count the total number of lines.

In [70]:
rdd_russia.count()

8

Count how many lines contain the word "Russia"

In [71]:
rdd_russia.filter(lambda line: "Russia" in line).count()

6

Find the most 5 frequent word in the file.

In [72]:
words_rdd = rdd_russia.flatMap(lambda line: line.lower().split())
word_counts = words_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
word_counts.takeOrdered(5, key=lambda x: -x[1])

[('the', 10), ('is', 7), ('in', 5), ('russia', 5), ('world', 3)]

Tokenize words

In [73]:
tokenized_words = rdd_russia.flatMap(lambda line: line.lower().split()).collect()
print(tokenized_words)

['russia', 'is', 'the', 'largest', 'country', 'in', 'the', 'world', 'by', 'land', 'area', 'moscow', 'is', 'the', 'capital', 'city', 'of', 'russia', 'the', 'russian', 'language', 'is', 'one', 'of', 'the', 'most', 'widely', 'spoken', 'languages', 'in', 'the', 'world', 'russia', 'is', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'the', 'trans-siberian', 'railway', 'is', 'the', 'longest', 'railway', 'line', 'in', 'the', 'world', 'russia', 'has', 'a', 'strong', 'tradition', 'in', 'literature,', 'music', 'and', 'ballet', 'the', 'country', 'is', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'is', 'a', 'major', 'player', 'in', 'global', 'energy', 'production']


Remove stopwords (a, the, is, to, in, of). 

In [74]:
stopwords = {'a', 'the', 'is', 'to', 'in', 'of'}
filtered_words = rdd_russia.flatMap(lambda line: line.lower().split())\
                          .filter(lambda word: word not in stopwords)\
                          .collect()
print(filtered_words)

['russia', 'largest', 'country', 'world', 'by', 'land', 'area', 'moscow', 'capital', 'city', 'russia', 'russian', 'language', 'one', 'most', 'widely', 'spoken', 'languages', 'world', 'russia', 'known', 'for', 'its', 'rich', 'history', 'and', 'culture', 'trans-siberian', 'railway', 'longest', 'railway', 'line', 'world', 'russia', 'has', 'strong', 'tradition', 'literature,', 'music', 'and', 'ballet', 'country', 'famous', 'for', 'its', 'cold', 'winters', 'and', 'vast', 'landscapes', 'russia', 'major', 'player', 'global', 'energy', 'production']


Count the frequency of each word

In [75]:
word_freq = rdd_russia.flatMap(lambda line: line.lower().split())\
                     .filter(lambda word: word not in stopwords)\
                     .map(lambda word: (word, 1))\
                     .reduceByKey(lambda a, b: a + b)\
                     .collect()
print(word_freq)

[('world', 3), ('by', 1), ('land', 1), ('history', 1), ('and', 3), ('winters', 1), ('spoken', 1), ('culture', 1), ('strong', 1), ('largest', 1), ('russian', 1), ('famous', 1), ('cold', 1), ('its', 2), ('ballet', 1), ('capital', 1), ('language', 1), ('most', 1), ('widely', 1), ('known', 1), ('line', 1), ('music', 1), ('energy', 1), ('city', 1), ('languages', 1), ('trans-siberian', 1), ('longest', 1), ('has', 1), ('major', 1), ('country', 2), ('area', 1), ('for', 2), ('player', 1), ('production', 1), ('moscow', 1), ('vast', 1), ('global', 1), ('literature,', 1), ('landscapes', 1), ('one', 1), ('railway', 2), ('tradition', 1), ('russia', 5), ('rich', 1)]


In [76]:
schema = 'id integer, name string, age integer, salary integer' 
data = [
    (1, "Ali", 25, 4000),
    (2, "Mariam", 30, 6000),
    (3, "Omar", 35, 7000),
    (4, "Sara", 28, 5000),
    (5, "Omar", 25, 6500),
    (6, "Mariam", 26, 7500)
]

df = spark.createDataFrame(data,schema)

Show schema and first 2 rows

In [77]:
df.printSchema()
df.show(2)

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  1|   Ali| 25|  4000|
|  2|Mariam| 30|  6000|
+---+------+---+------+
only showing top 2 rows



Select only name and salary

In [78]:
df.select("name", "salary").show()

+------+------+
|  name|salary|
+------+------+
|   Ali|  4000|
|Mariam|  6000|
|  Omar|  7000|
|  Sara|  5000|
|  Omar|  6500|
|Mariam|  7500|
+------+------+



Find the average salary

In [79]:
from pyspark.sql.functions import avg
df.select(avg("salary")).show()

+-----------+
|avg(salary)|
+-----------+
|     6000.0|
+-----------+



Filter employees older than 28

In [80]:
df.filter(df.age > 28).show()

+---+------+---+------+
| id|  name|age|salary|
+---+------+---+------+
|  2|Mariam| 30|  6000|
|  3|  Omar| 35|  7000|
+---+------+---+------+



Count distinct values in the name column

In [81]:
df.select("name").distinct().count()

4

Group by a the name column and find average salary

In [82]:
from pyspark.sql.functions import avg
df.groupBy("name").agg(avg("salary").alias("Average")).show()

+------+-------+
|  name|Average|
+------+-------+
|   Ali| 4000.0|
|Mariam| 6750.0|
|  Omar| 6750.0|
|  Sara| 5000.0|
+------+-------+



In [83]:

from pyspark.sql import Row

data = [
    Row(Id="emp1", Name="John", Sales=None),
    Row(Id="emp2", Name=None, Sales=None),
    Row(Id="emp3", Name=None, Sales=345.0),
    Row(Id="emp4", Name="Cindy", Sales=456.0)
]



In [84]:
df1 = spark.createDataFrame(data)
df1.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [85]:
# df1 = spark.read.csv("/data/NullData.csv", header=True, inferSchema=True) #this file in shared folder
# df1.show()

Find the avg sales 

In [86]:
from pyspark.sql.functions import avg
avg_sales = df1.select(avg("Sales")).collect()[0][0]
print(avg_sales)

400.5


Replace null name with 'Unknown' and sales with the avg sales of the column 

In [87]:
from pyspark.sql.functions import when, col, lit

df_cleaned = df1.withColumn("Name", when(col("Name").isNull(), "Unknown").otherwise(col("Name")))\
                .withColumn("Sales", when(col("Sales").isNull(), avg_sales).otherwise(col("Sales")))
df_cleaned.show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John|400.5|
|emp2|Unknown|400.5|
|emp3|Unknown|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+

